# Autoshadow

## Process Overview

1. Separate the oversize paths
1. For each oversize path:
    1. Discretize into points and turn into polygons
    1. Calculate the angle of each polygon segment. Record this and create a histogram of the most popular angles. Create a histogram for each path, and a global histogram.
    1. For the most popular angles, draw a line from each point and calculate the length of cut that it would make through the path, and the maximum width of the bounding box of the geometries created by a cut at that point. Remember to take compound shapes into consideration.
    1. Create a score based on:
        1. the angle's histogram score (with adjustable weight between the global histogram and a histogram of this path alone);
        1. the widths of the generated paths (shorter is better);
        1. the length of cut (shorter is better); and
        1. the rate of change at that point (corners are preferable).
    1. Divide the object along that line.
    1. Remove any paths that are now below the maximum size and repeat the process with the remaining oversize paths.

All paths are now less than or equal to the maximum shadow width.

1. Remove any duplicate paths.
1. Remove any paths that are wholly contained within another path.
1. For every path, calculate the maximum width of paths that could covered by a shadow that starts at that path. Once the maximum area covered is found, merge all of the paths within that area into a single path and remove from the list. Repeat until the list is empty.

## Code

In [8]:
import io
import numpy as np
import matplotlib.pyplot as plt
import colorsys
from svgelements import *
from ipywidgets import interactive 
from ipywidgets import *
from IPython.core import display
import itertools
import math

class Autoshadow:
    """Automatically optimise shadow positions"""


    def __init__(self, maxwidth=1, tolerance=0):
        self.paths = []
        self.shadows = []
        self.moved_shadows = []
        self.inputfile = None
        self.maxwidth = maxwidth
        self.tolerance = tolerance

    def bail(self, errorMessage):
        print(errorMessage, file=sys.stderr)
        sys.exit(1)

    """ Getters and Setters """
    
    def getPaths(self):
        return self.paths

    def getShadows(self):
        return self.shadows

    def setPaths(self, paths):
        """Setter for the list of paths, mostly used for testing"""
        self.paths = paths

    def setShadows(self, shadows):
        """Setter for the list of shadows, mostly used for testing"""
        self.shadows = shadows
        
    def setMaxWidth(self, new_max_width):
        self.maxwidth = new_max_width

    """ Measurements """
    
    def totalWidth(self, path1, path2):
        paths = self.sortPathsByStartPosition([path1, path2])
        return float(paths[1][0]) + float(paths[1][1]) - float(paths[0][0])
    
    """ Sorting """
    
    def sortPathsByWidth(self, paths):
        def pathWidth(path):
            return float(path[1])

        paths.sort(key=pathWidth, reverse=True)
        return paths

    def sortPathsByStartPosition(self, paths):
        def pathStart(path):
            return float(path[0])

        paths.sort(key=pathStart)
        return paths

    def removeDuplicates(self, listOfLists):
        listOfLists.sort()
        return list(
            listOfLists for listOfLists, _ in itertools.groupby(listOfLists)
        )

    def getRawPathData(self):
        # TODO: Merge all paths with the same name
        raw = []
        for path in self.paths:
            raw.append([float(path[1]), float(path[3])])
        return raw

    def removeOversizePaths(self, paths):
        sizedPaths = []
        for path in paths:
            if float(path[1]) <= float(self.maxwidth):
                sizedPaths.append(path)

        return sizedPaths

    """ Math Functions """
    
    def round_up(self, number: float, decimals: int = 2):
        """Returns a value rounded up to a specific number of decimal places."""
        if not isinstance(decimals, int):
            raise TypeError("decimal places must be an integer")
        elif decimals < 0:
            raise ValueError("decimal places has to be 0 or more")
        elif decimals == 0:
            return math.ceil(number)

        factor = 10 ** decimals
        return math.ceil((number + self.tolerance) * factor) / factor

    def round_down(self, number: float, decimals: int = 2):
        """
        Returns a value rounded down to a specific number of decimal places.
        """
        if not isinstance(decimals, int):
            raise TypeError("decimal places must be an integer")
        elif decimals < 0:
            raise ValueError("decimal places has to be 0 or more")
        elif decimals == 0:
            return math.floor(number)

        factor = 10 ** decimals
        return math.floor((number - self.tolerance) * factor) / factor

    def roundPaths(self, paths):
        # Ensure that paths are rounded up to avoid cropping
        roundedPaths = []
        for path in paths:
            roundedPaths.append(
                [self.round_down(path[0], 2), self.round_up(path[1], 2)]
            )

        return roundedPaths

    """ Strategies """
    
    def removeInternalPaths(self, paths):
        majorPaths = []
        for i in range(0, len(paths)):
            is_subpath = False
            for j in range(0, len(paths)):
                if i == j:
                    continue
                # if i exists within j then i should be dropped
                if (float(paths[i][0]) >= float(paths[j][0])) and (
                    float(paths[i][0]) + float(paths[i][1])
                ) <= (float(paths[j][0]) + float(paths[j][1])):
                    if paths[i] != paths[j]:
                        # print("Subpath", paths[i], "exists within", paths[j], file=sys.stderr)
                        is_subpath = True
                        break
                # else:
                #     print("Path", paths[i], "is not within", paths[j], "or is identical", file=sys.stderr)
            if is_subpath is False:
                majorPaths.append(paths[i])

        return majorPaths

    def removeOverlappingPaths(self, paths):
        try:
            nonOverlappingPaths = [paths[0]]
        except IndexError:
            return paths
        # print(paths, file=sys.stderr)
        for i in range(1, len(paths)):
            # print("Processing", i, file=sys.stderr)
            badshadow = False
            for j in range(0, len(nonOverlappingPaths)):
                # If start of j is before end of i
                # print("j:", j, paths[j], file=sys.stderr)
                # print(float(paths[j][0]), float(paths[i][0]), float(paths[i][1]), file=sys.stderr)
                if float(paths[i][0]) < (
                    float(nonOverlappingPaths[j][0])
                    + float(nonOverlappingPaths[j][1])
                ):
                    # If start position is before the end of the previous shadow, it's bad
                    badshadow = True
                    # print("Path", paths[i], "overlaps", paths[j], "(", float(paths[i][0]), "<", (float(nonOverlappingPaths[j][0]) + float(nonOverlappingPaths[j][1])), file=sys.stderr)
                    break
            if badshadow is False:
                nonOverlappingPaths.append(paths[i])

        return nonOverlappingPaths

    def mergeNeighbouringPaths(self, paths):
        unduplicatePaths = self.removeDuplicates(paths)
        mergedPaths = []
        for i in range(0, len(unduplicatePaths)):
            idxBestMerge = -1
            valBestMerge = 0
            idxNextTry = -1
            for j in range(i + 1, len(unduplicatePaths)):
                width = self.totalWidth(
                    unduplicatePaths[i], unduplicatePaths[j]
                )
                if width > valBestMerge and width <= float(self.maxwidth):
                    # print("New best width of", unduplicatePaths[i], "and", unduplicatePaths[j], "is", width, file=sys.stderr)
                    # Found a new widest pair
                    idxBestMerge = i
                    valBestMerge = width
                    idxNextTry = j + 1
            if idxBestMerge > -1:
                mergedPaths.append(
                    [
                        float(unduplicatePaths[idxBestMerge][0]),
                        float(valBestMerge),
                    ]
                )
                i = idxNextTry
            else:
                mergedPaths.append(
                    [unduplicatePaths[i][0], unduplicatePaths[i][1]]
                )
                # print("Storing best path:", unduplicatePaths[idxBestMerge][0], float(valBestMerge), file=sys.stderr)

        return mergedPaths

    def subdividePath(self, path, existingPaths):
        
        # For now, just split the path along any existing shadows.
        # If any remaining piece is still larger than the max width,
        # then divide it equally.
        
        #for path in all_paths_but_this_one
        #if path start covers a part of big path
        #    split big path at path start
        #if path end covers a part of big path
        #    split big path at path end     
        return path

    def breakLargePaths(self, paths):
        """ This needs to be run after all other paths have been created,
            so that we can reuse those paths """
        brokenPaths = []
        for i in range(0, len(paths)):
            if (float(paths[i][1] > self.maxwidth)):
                subdividedPath = self.subdividePath(paths[i])
                brokenPaths.append(subdividedPath)
            else:
                brokenPaths.append(paths[i])
        
        return brokenPaths

    def filterOversizePaths(self, paths):
        
        return paths, []

    def calculateOptimalPaths(self, paths, max_width=None):

        if max_width is not None:
            self.maxwidth = max_width
    
        # Sort paths by size
        sortedPaths = self.sortPathsByWidth(paths)

        sizedPaths, oversizePaths = self.filterOversizePaths(paths)

        # Merge all possible path combinations
        mergedPaths = self.removeDuplicates(
            self.mergeNeighbouringPaths(sizedPaths)
        )

        # Remove overlapping merged paths
        nooverlapMergedPaths = self.removeOverlappingPaths(
            self.sortPathsByStartPosition(mergedPaths)
        )

        # Remove paths that exist inside other paths
        majorPaths = self.roundPaths(
            self.removeInternalPaths(
                self.sortPathsByStartPosition(nooverlapMergedPaths)
            )
        )
        
        # Split paths larger than max
        splitPaths = self.breakLargePaths(oversizePaths)

        # Remove duplicates
        return self.removeDuplicates(majorPaths)

    
class SvgParser:
    def __init__(self, filename):
        #svg_stream = io.StringIO(svg_string)
        self.parsed_svg = svgelements.SVG.parse(filename, transform='Matrix(1, 0, 0, -1, 0, 0)')
        #print(parsed_svg)
        #svg_stream.close()
        self.paths = []
        self.shadows = []
        self.moved_shadows = []

    def parse_paths(self, svg):

        elements = []

        for element in svg.elements():
            try:
                if element.values['visibility'] == 'hidden':
                    continue
            except (KeyError, AttributeError):
                pass
            if isinstance(element, SVGText):
                elements.append(element)
            elif isinstance(element, Path):
                if len(element) != 0:
                    elements.append(element)
            elif isinstance(element, Shape):
                e = Path(element)
                e.reify()  # In some cases the shape could not have reified, the path must.
                if len(e) != 0:
                    elements.append(e)

        return elements

    def separate_paths_by_size(self, paths, maximum_width):

        regular_paths, oversize_paths = [], []
        oversize_total_length = 0

        for path in paths:
            #print(path, type(path))
            if path is None:
                continue
            x1, y1, x2, y2 = path.bbox()
            #print("x1 y1 x2 y2 width", x1, y1, x2, y2, x2 - x1)
            if x2 - x1 > maximum_width:
                #print("Adding to oversize paths")
                oversize_paths.append(path)
                oversize_total_length = oversize_total_length + path.length()
            else:
                #print("Adding to regular paths")
                regular_paths.append(path)

        return regular_paths, oversize_paths, oversize_total_length

    # Use svgelements to discretise path

    def get_path_xpos_and_width(self, paths):

        xpos_and_width = []

        for path in paths:
            x1, y1, x2, y2 = path.bbox()
            xpos_and_width.append([x1, x2-x1])

        return xpos_and_width

    def discretise(self, path, num_points, oversize_total_length):

        x_arr   = []
        y_arr   = []
        col_arr = []

        # Obtain this path's proportion of all lengths combined so that we can 
        # calculate it's share of the total points available
        path_points = (path.length() / oversize_total_length) * num_points
        for i in np.arange(0,1,1/path_points):
            #print("discretise: ", i)
            x, y = path.point(i)
            x_arr.append(x)
            y_arr.append(y)
            col_arr.append('#000000')
            #print(len(x_arr), len(y_arr), len(col_arr))

        return x_arr, y_arr, col_arr

    # Calculate angle and derivative

    def calculate_derivatives(self, points_x, points_y):

        # Since the first point needs info from the last point and vice-versa,
        # we copy the array and duplicate those points

        drv_points_x = points_x.copy()
        drv_points_x.insert(0, points_x[-1])
        drv_points_x.append(points_x[0])

        drv_points_y = points_y.copy()
        drv_points_y.insert(0, points_y[-1])
        drv_points_y.append(points_y[0])

        derivatives = []
        for i in range(1, len(drv_points_x)-1):
            xDiff = drv_points_x[i+1] - drv_points_x[i-1]
            yDiff = drv_points_y[i+1] - drv_points_y[i-1]
            derivatives.append(degrees(atan2(yDiff, xDiff))%180)

        return derivatives   

    # Calculate histograms

    # numpy.histogram_bin_edges()



In [12]:
# Interactive Plot

def myplot(svgparser, num_points, max_width):

    regular_paths, oversize_paths, oversize_total_length = svgparser.separate_paths_by_size(svgparser.parse_paths(svgparser.parsed_svg), max_width)

    plt.rcParams['figure.figsize'] = [15, 10]

    fig, axes = plt.subplots(nrows=2, ncols=1)
    ax0, ax1 = axes.flatten()
    ax0.set_aspect('equal')

    all_derivatives = []
    for path in oversize_paths:
        x, y, col = svgparser.discretise(path.reify(), num_points, oversize_total_length)
        derivatives = svgparser.calculate_derivatives(x, y)
        colours = []
        for derivative in derivatives:
            all_derivatives.append(derivative)
            colour = colorsys.hls_to_rgb(derivative / 180, 0.40, 1.0)
            colstr = (
                "#"
                + format(int(colour[0] * 255), "02x")
                + format(int(colour[1] * 255), "02x")
                + format(int(colour[2] * 255), "02x")
            )
            colours.append(colstr)
        ax0.scatter(x, y, c = colours)

    edges = np.histogram_bin_edges(all_derivatives, bins=180)
    ax1.hist(all_derivatives, bins=edges)

    fig.tight_layout()
    plt.show()

plot = SvgParser(filename = 'autoshadow_test.svg')
interactive(myplot,
            svgparser = fixed(plot),
            num_points = widgets.FloatSlider(min=10, max=1000, step=10, value=200, continuous_update=True), 
            max_width = widgets.FloatSlider(min=0.5, max=10, value=0.5, continuous_update=True))

interactive(children=(FloatSlider(value=200.0, description='num_points', max=1000.0, min=10.0, step=10.0), Flo…

interactive(children=(FloatSlider(value=200.0, description='num_points', max=1000.0, min=10.0, step=10.0), Flo…

<div class="alert alert-block alert-warning">
    <b>Note:</b> Edit the SVG file and change the width and height to "1mm", and the viewbox to "0 0 1 1", to ensure that the units are correctly calculated. 
</div>

## Existing Autoshadow code

## Interactive Chart

In [6]:
shadow = Autoshadow(maxwidth=1, tolerance=0)

# Paths are in the format x, width
shadow.setPaths([
            [10, 1.5],
            [11, 1.5],
            [12, 1.5],
            [13, 1.5],
            [10.5, 1]
])

widgets.interact(shadow.calculateOptimalPaths, paths=fixed(shadow.getPaths()), max_width=widgets.FloatSlider(min=0, max=5, step=0.1, value=1));

# Sliders: max width, number of histogram buckets, number of points to discretise,

interactive(children=(FloatSlider(value=1.0, description='max_width', max=5.0), Output()), _dom_classes=('widg…

In [ ]:
%matplotlib widget
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import SVG, display
from skimage.draw import ellipse
from skimage.measure import find_contours, approximate_polygon, subdivide_polygon

from io import StringIO  # To convert string to csv file
import csv

import re
import xml.etree.ElementTree as ET
import getopt
import math
    
class Autoshadow:
    """Automatically optimise shadow positions"""


    def __init__(self, maxwidth=1, tolerance=0):
        self.paths = []
        self.shadows = []
        self.moved_shadows = []
        self.inputfile = None
        self.maxwidth = maxwidth
        self.tolerance = tolerance

    def bail(self, errorMessage):
        print(errorMessage, file=sys.stderr)
        sys.exit(1)

    """ Getters and Setters """
    
    def getPaths(self):
        return self.paths

    def getShadows(self):
        return self.shadows

    def setPaths(self, paths):
        """Setter for the list of paths, mostly used for testing"""
        self.paths = paths

    def setShadows(self, shadows):
        """Setter for the list of shadows, mostly used for testing"""
        self.shadows = shadows
        
    def setMaxWidth(self, new_max_width):
        self.maxwidth = new_max_width

    """ Measurements """
    
    def totalWidth(self, path1, path2):
        paths = self.sortPathsByStartPosition([path1, path2])
        return float(paths[1][0]) + float(paths[1][1]) - float(paths[0][0])
    
    """ Sorting """
    
    def sortPathsByWidth(self, paths):
        def pathWidth(path):
            return float(path[1])

        paths.sort(key=pathWidth, reverse=True)
        return paths

    def sortPathsByStartPosition(self, paths):
        def pathStart(path):
            return float(path[0])

        paths.sort(key=pathStart)
        return paths

    def removeDuplicates(self, listOfLists):
        listOfLists.sort()
        return list(
            listOfLists for listOfLists, _ in itertools.groupby(listOfLists)
        )

    def getRawPathData(self):
        # TODO: Merge all paths with the same name
        raw = []
        for path in self.paths:
            raw.append([float(path[1]), float(path[3])])
        return raw

    def removeOversizePaths(self, paths):
        sizedPaths = []
        for path in paths:
            if float(path[1]) <= float(self.maxwidth):
                sizedPaths.append(path)

        return sizedPaths

    """ Math Functions """
    
    def round_up(self, number: float, decimals: int = 2):
        """Returns a value rounded up to a specific number of decimal places."""
        if not isinstance(decimals, int):
            raise TypeError("decimal places must be an integer")
        elif decimals < 0:
            raise ValueError("decimal places has to be 0 or more")
        elif decimals == 0:
            return math.ceil(number)

        factor = 10 ** decimals
        return math.ceil((number + self.tolerance) * factor) / factor

    def round_down(self, number: float, decimals: int = 2):
        """
        Returns a value rounded down to a specific number of decimal places.
        """
        if not isinstance(decimals, int):
            raise TypeError("decimal places must be an integer")
        elif decimals < 0:
            raise ValueError("decimal places has to be 0 or more")
        elif decimals == 0:
            return math.floor(number)

        factor = 10 ** decimals
        return math.floor((number - self.tolerance) * factor) / factor

    def roundPaths(self, paths):
        # Ensure that paths are rounded up to avoid cropping
        roundedPaths = []
        for path in paths:
            roundedPaths.append(
                [self.round_down(path[0], 2), self.round_up(path[1], 2)]
            )

        return roundedPaths

    """ Strategies """
    
    def removeInternalPaths(self, paths):
        majorPaths = []
        for i in range(0, len(paths)):
            is_subpath = False
            for j in range(0, len(paths)):
                if i == j:
                    continue
                # if i exists within j then i should be dropped
                if (float(paths[i][0]) >= float(paths[j][0])) and (
                    float(paths[i][0]) + float(paths[i][1])
                ) <= (float(paths[j][0]) + float(paths[j][1])):
                    if paths[i] != paths[j]:
                        # print("Subpath", paths[i], "exists within", paths[j], file=sys.stderr)
                        is_subpath = True
                        break
                # else:
                #     print("Path", paths[i], "is not within", paths[j], "or is identical", file=sys.stderr)
            if is_subpath is False:
                majorPaths.append(paths[i])

        return majorPaths

    def removeOverlappingPaths(self, paths):
        try:
            nonOverlappingPaths = [paths[0]]
        except IndexError:
            return paths
        # print(paths, file=sys.stderr)
        for i in range(1, len(paths)):
            # print("Processing", i, file=sys.stderr)
            badshadow = False
            for j in range(0, len(nonOverlappingPaths)):
                # If start of j is before end of i
                # print("j:", j, paths[j], file=sys.stderr)
                # print(float(paths[j][0]), float(paths[i][0]), float(paths[i][1]), file=sys.stderr)
                if float(paths[i][0]) < (
                    float(nonOverlappingPaths[j][0])
                    + float(nonOverlappingPaths[j][1])
                ):
                    # If start position is before the end of the previous shadow, it's bad
                    badshadow = True
                    # print("Path", paths[i], "overlaps", paths[j], "(", float(paths[i][0]), "<", (float(nonOverlappingPaths[j][0]) + float(nonOverlappingPaths[j][1])), file=sys.stderr)
                    break
            if badshadow is False:
                nonOverlappingPaths.append(paths[i])

        return nonOverlappingPaths

    def mergeNeighbouringPaths(self, paths):
        unduplicatePaths = self.removeDuplicates(paths)
        mergedPaths = []
        for i in range(0, len(unduplicatePaths)):
            idxBestMerge = -1
            valBestMerge = 0
            idxNextTry = -1
            for j in range(i + 1, len(unduplicatePaths)):
                width = self.totalWidth(
                    unduplicatePaths[i], unduplicatePaths[j]
                )
                if width > valBestMerge and width <= float(self.maxwidth):
                    # print("New best width of", unduplicatePaths[i], "and", unduplicatePaths[j], "is", width, file=sys.stderr)
                    # Found a new widest pair
                    idxBestMerge = i
                    valBestMerge = width
                    idxNextTry = j + 1
            if idxBestMerge > -1:
                mergedPaths.append(
                    [
                        float(unduplicatePaths[idxBestMerge][0]),
                        float(valBestMerge),
                    ]
                )
                i = idxNextTry
            else:
                mergedPaths.append(
                    [unduplicatePaths[i][0], unduplicatePaths[i][1]]
                )
                # print("Storing best path:", unduplicatePaths[idxBestMerge][0], float(valBestMerge), file=sys.stderr)

        return mergedPaths

    def subdividePath(self, path, existingPaths):
        
        # For now, just split the path along any existing shadows.
        # If any remaining piece is still larger than the max width,
        # then divide it equally.
        
        #for path in all_paths_but_this_one
        #if path start covers a part of big path
        #    split big path at path start
        #if path end covers a part of big path
        #    split big path at path end     
        return path

    def breakLargePaths(self, paths):
        """ This needs to be run after all other paths have been created,
            so that we can reuse those paths """
        brokenPaths = []
        for i in range(0, len(paths)):
            if (float(paths[i][1] > self.maxwidth)):
                subdividedPath = self.subdividePath(paths[i])
                brokenPaths.append(subdividedPath)
            else:
                brokenPaths.append(paths[i])
        
        return brokenPaths

    def filterOversizePaths(self, paths):
        
        return paths, []

    def calculateOptimalPaths(self, paths, max_width=None):

        if max_width is not None:
            self.maxwidth = max_width
    
        # Sort paths by size
        sortedPaths = self.sortPathsByWidth(paths)

        sizedPaths, oversizePaths = self.filterOversizePaths(paths)

        # Merge all possible path combinations
        mergedPaths = self.removeDuplicates(
            self.mergeNeighbouringPaths(sizedPaths)
        )

        # Remove overlapping merged paths
        nooverlapMergedPaths = self.removeOverlappingPaths(
            self.sortPathsByStartPosition(mergedPaths)
        )

        # Remove paths that exist inside other paths
        majorPaths = self.roundPaths(
            self.removeInternalPaths(
                self.sortPathsByStartPosition(nooverlapMergedPaths)
            )
        )
        
        # Split paths larger than max
        splitPaths = self.breakLargePaths(oversizePaths)

        # Remove duplicates
        return self.removeDuplicates(majorPaths)

## Example Charts

In [ ]:
from ipywidgets import *
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [15, 10]

def interactive_multiplot(a):
    # A more complex example from the matplotlib gallery
    np.random.seed(0)

    n_bins = int(a)
    x = np.random.randn(1000, 3)

    fig, axes = plt.subplots(nrows=2, ncols=2)
    ax0, ax1, ax2, ax3 = axes.flatten()

    colors = ['red', 'tan', 'lime']
    ax0.hist(x, n_bins, density=1, histtype='bar', color=colors, label=colors)
    ax0.legend(prop={'size': 10})
    ax0.set_title('bars with legend')

    ax1.hist(x, n_bins, density=1, histtype='bar', stacked=True)
    ax1.set_title('stacked bar')

    ax2.hist(x, n_bins, histtype='step', stacked=True, fill=False)
    ax2.set_title('stack step (unfilled)')

    # Make a multiple-histogram of data-sets with different length.
    x_multi = [np.random.randn(n) for n in [10000, 5000, 2000]]
    ax3.hist(x_multi, n_bins, histtype='bar')
    ax3.set_title('different sample sizes')

    fig.tight_layout()
    plt.show()

interactive(interactive_multiplot,
            a = widgets.FloatSlider(min=5, max=100, step=1, value=10, continuous_update=False))

## Unit Tests

In [ ]:
import unittest

def add(a, b):
    return a + b

class TestNotebook(unittest.TestCase):
    
    def test_add(self):
        self.assertEqual(add(2, 2), 4)
        

unittest.main(argv=[''], verbosity=2, exit=False);

<div class="alert alert-block alert-info">
<b>Tip:</b> Use blue boxes (alert-info) for tips and notes. 
If it’s a note, you don’t have to include the word “Note”.
</div>